In [336]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import RobustScaler
import pandas as pd
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_columns', None)

In [337]:
df=pd.read_parquet('../data/processed/df.parquet')
dfd=pd.read_parquet('../data/processed/dfd.parquet')


In [338]:
dfd.columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'df_original', 'id',
       'cut_int', 'color_int', 'clarity_int', 'cut_color_clarity_score',
       'raw_density', 'marketing_carat_int', 'carat_sqr', 'cut_Fair',
       'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D',
       'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
       'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2', 'marketing_carat_bad',
       'marketing_carat_good', 'marketing_carat_regular'],
      dtype='object')

In [339]:
#option 1

# dfd=dfd.drop(columns=['marketing_carat_int','cut_int','color_int', 'clarity_int'])

# #option 2
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score'])

# #option 3 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z'])

# #option 3 bis espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density'])

# #option 3 bis2 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'x', 'y', 'z'])

# #option 4
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z','raw_density'])

# #option 5 SIN NADA
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density','marketing_carat_bad',
#        'marketing_carat_good', 'marketing_carat_regular'])

#option 5 SIN NADA
dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
                     'marketing_carat_bad',
       'marketing_carat_good', 'marketing_carat_regular','carat_sqr'])

In [340]:
#option 1

# df=df.drop(columns=['cut','color', 'clarity','marketing_carat'])

# #option 2
# df=df.drop(columns=['cut','color', 'clarity','marketing_carat','cut_color_clarity_score'])

# #option 2

df=df.drop(columns=['cut','color', 'clarity','marketing_carat','marketing_carat_int',
                       'carat_sqr'])

# df=df.drop(columns=['cut','color', 'clarity','marketing_carat',
#                        'carat_sqr'])

# df=df.drop(columns=['cut','color', 'clarity','marketing_carat','marketing_carat_int','raw_density',
#                        'carat_sqr'])

In [341]:
df


,carat,depth,table,price,x,y,z,df_original,id,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density
0,1.21,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,4,1,4,31,3.683471
1,0.32,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,3,3,4,34,3.664419
2,0.71,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,1,4,5,35,3.755392
3,0.41,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,2,7,3,39,3.712154
4,1.02,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,5,4,3,39,3.633558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,NaN,5.35,5.32,3.30,0,13480.0,5,6,3,45,3.641219
13481,0.71,62.2,55.0,NaN,5.71,5.73,3.56,0,13481.0,5,2,4,37,3.657370
13482,0.70,61.6,55.0,NaN,5.75,5.71,3.53,0,13482.0,5,5,5,50,3.623853
13483,0.70,58.8,57.0,NaN,5.85,5.89,3.45,0,13483.0,3,5,2,32,3.533125


In [342]:
dfd

,carat,depth,table,price,x,y,z,df_original,id,cut_color_clarity_score,raw_density,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,62.4,58.0,4268.0,6.83,6.79,4.25,1,NaN,31,3.683471,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,0.32,63.0,57.0,505.0,4.35,4.38,2.75,1,NaN,34,3.664419,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,0.71,65.5,55.0,2686.0,5.62,5.53,3.65,1,NaN,35,3.755392,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
3,0.41,63.8,56.0,738.0,4.68,4.72,3.00,1,NaN,39,3.712154,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1.02,60.5,59.0,4882.0,6.55,6.51,3.95,1,NaN,39,3.633558,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,61.9,56.0,NaN,5.35,5.32,3.30,0,13480.0,45,3.641219,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
13481,0.71,62.2,55.0,NaN,5.71,5.73,3.56,0,13481.0,37,3.657370,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
13482,0.70,61.6,55.0,NaN,5.75,5.71,3.53,0,13482.0,50,3.623853,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
13483,0.70,58.8,57.0,NaN,5.85,5.89,3.45,0,13483.0,32,3.533125,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


## Split

In [343]:


df_final=df

data=df_final.loc[df['df_original']==1]

############## FILAS


data=data.drop_duplicates(subset=data.columns, keep='first')

data['depth_calc'] = data['z'] / ((data['x'] + data['y'])/2)*100
data['rest']=data['depth_calc'] - data['depth'] 
filter_rest=(data['rest']<-1) | (data['rest']>1)
data=data.loc[~filter_rest]
data=data.drop(columns=['depth_calc','rest'],axis=1)

filter_xyz=(data['x']<0.2) | (data['y']<0.2) | (data['z']<0.2)
data=data.loc[~filter_xyz ]



df_predict=df_final.loc[df['df_original']==0]

X=data.drop(columns=['price','df_original','id'])
Y=data['price']

X_predict=df_predict.drop(columns=['price','df_original','id'])


rs = RobustScaler() 
X = pd.DataFrame(rs.fit_transform(X), columns= X.columns)
X_predict = pd.DataFrame(rs.fit_transform(X_predict), columns= X_predict.columns)

X.to_csv('../data/processed/X.csv')
Y.to_csv('../data/processed/Y.csv')
X_predict.to_csv('../data/processed/X_predict.csv')



print(data.shape)
print(df_predict.shape)
print(X.shape)
print(X_predict.shape)



(40292, 14)
(13485, 14)
(40292, 11)
(13485, 11)


## Model Generator

In [345]:
# arboles  de decision max_depth probar min sample split


estimators=[ ('RandomForestRegressor', RandomForestRegressor(max_depth=50, min_samples_split=8, n_estimators=100, random_state=0)),
             ('ExtraTreesRegressor',ExtraTreesRegressor(n_estimators=500,max_depth=16,n_jobs = -1, random_state=0)),
           ('GradientBoostingRegressor',GradientBoostingRegressor(random_state=0))]



machine = [RandomForestRegressor(max_depth=50, min_samples_split=8, n_estimators=100, random_state=0),
           ExtraTreesRegressor(n_estimators=500,max_depth=16,n_jobs = -1, random_state=0),
           GradientBoostingRegressor(random_state=0),
          StackingRegressor(estimators=estimators,final_estimator= LinearRegression(n_jobs = -1),cv= 5 , verbose = 10 , passthrough = True)]  


# machine = [   RandomForestRegressor(max_depth=50, min_samples_split=8, n_estimators=100, random_state=0) ]

# max_depth=50, min_samples_split=8, n_estimators=100

for x in machine:
    print(f'Model in action: {x}\n')
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.9999999999, random_state=32)
    
######################           ENTRENAR AL FINAL CON TODO


    
#     features = RFECV(x)
#     features= features.fit(X_train, Y_train)
#     features_columns=[X.columns[pos] for pos,item in enumerate(features.support_.tolist()) if item==False]
#     X_train=X_train.drop(columns=features_columns)
#     X_test=X_test.drop(columns=features_columns)
#     X_predict=df_predict.drop(columns=['price','df_original','id'])
#     X_predict=X_predict.drop(columns=features_columns)
#     print(f'Delete columns: {len(X.columns)-features.n_features_}  {features_columns} ')
        
        
    model = x  
    model.fit(X_train, Y_train)
    predictions_train = model.predict(X_train)#.clip(326,18823)
    predictions_test = model.predict(X_test)#.clip(326,18823)

    print(f' R2 Train: {r2_score(y_true=Y_train, y_pred=predictions_train)}')
    print(f' R2 Test: {r2_score(y_true=Y_test, y_pred=predictions_test)}')
    print(f' RMSE = {(mean_squared_error(Y_test, predictions_test,squared=False))}\n\n')
    









Model in action: RandomForestRegressor(max_depth=50, min_samples_split=8, random_state=0)

 R2 Train: 0.9945779703175648
 R2 Test: nan
 RMSE = 788.1088591408616


Model in action: ExtraTreesRegressor(max_depth=16, n_estimators=500, n_jobs=-1, random_state=0)



/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


 R2 Train: 0.9960137680525031
 R2 Test: nan
 RMSE = 774.0219727297635


Model in action: GradientBoostingRegressor(random_state=0)



/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


 R2 Train: 0.9797364494798526
 R2 Test: nan
 RMSE = 462.6569634899688


Model in action: StackingRegressor(cv=5,
                  estimators=[('RandomForestRegressor',
                               RandomForestRegressor(max_depth=50,
                                                     min_samples_split=8,
                                                     random_state=0)),
                              ('ExtraTreesRegressor',
                               ExtraTreesRegressor(max_depth=16,
                                                   n_estimators=500, n_jobs=-1,
                                                   random_state=0)),
                              ('GradientBoostingRegressor',
                               GradientBoostingRegressor(random_state=0))],
                  final_estimator=LinearRegression(n_jobs=-1), passthrough=True,
                  verbose=10)



/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   37.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   52.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   19.4

 R2 Train: 0.995061914710349
 R2 Test: nan
 RMSE = 824.7328777807361




/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/sklearn/metrics/_regression.py:594: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## RandomizedSearchCV

In [ ]:


model2= RandomForestRegressor()


param_grid = {'bootstrap': [True, False],
    'n_estimators': [16, 32, 64, 128, 256, 512, 750, 1000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 8, 10, 12],
    'max_depth' : [2, 4, 8, 16, 32, 64]
}

grid_search = RandomizedSearchCV(model2, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(X_train, Y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [28]:
grid_search.best_score_

-565.6027945108959

## PROBAR RESULTADO

In [319]:
dfX_test=pd.DataFrame(X_test)
dfY_test=pd.DataFrame(Y_test)
dfpredictions_test=pd.DataFrame(predictions_test)
dftest=pd.concat([dfX_test.reset_index(drop=True),dfY_test.reset_index(drop=True),dfpredictions_test.reset_index(drop=True)], axis=1) 
dftest.rename(columns={0: 'price_predict'},inplace=True)
dftest['Diff']=dftest['price']-dftest['price_predict']

print(dftest['Diff'].describe())
dftest.loc[dftest['Diff']>2000].describe()

count      1.000000
mean     859.061216
std             NaN
min      859.061216
25%      859.061216
50%      859.061216
75%      859.061216
max      859.061216
Name: Diff, dtype: float64


,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,price,price_predict,Diff
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [352]:
# #####################importante  y_predict.clip(326,18823) min y maximo
# scores = cross_val_score(model, 
#                          diamonds[FEATS], 
#                          diamonds[TARGET], 
#                          scoring='neg_root_mean_squared_error', 
#                          cv=5, n_jobs=-1)
#para validarlo pero peta el ordenador

predictions_competition = model.predict(X_predict).clip(300,22000)
submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition})
submission['id'] = submission['id'].astype('int64')

submission.to_csv('/home/carpiero/IRONHACK/kaggle/submission.csv', index=False)

PROBA=pd.concat([X_predict,submission], axis=1)



submission


,id,price
0,0,3002.327753
1,1,5509.484037
2,2,9554.926027
3,3,4055.000556
4,4,1713.702089
...,...,...
13480,13480,1644.425414
13481,13481,2353.353648
13482,13482,2932.391791
13483,13483,2121.477868


In [347]:
PROBA.shape

(13485, 13)

In [348]:
PROBA

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,id,price
0,0.140625,0.533333,1.000000,0.066667,0.094444,0.125000,-0.5,0.333333,-0.5,-0.384615,1.145788,0,3002.327753
1,0.781250,-0.600000,0.000000,0.616667,0.650000,0.580357,0.5,-1.000000,0.5,-0.230769,0.007319,1,5509.484037
2,1.359375,0.200000,1.333333,0.933333,0.888889,0.928571,0.0,-0.333333,-0.5,-0.615385,1.710806,2,9554.926027
3,0.312500,1.266667,-1.000000,0.216667,0.227778,0.330357,-0.5,0.333333,-0.5,-0.384615,0.453914,3,4055.000556
4,-0.312500,0.666667,0.333333,-0.361111,-0.350000,-0.303571,-0.5,0.333333,0.5,0.230769,-0.138673,4,1713.702089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,-0.203125,0.000000,-0.333333,-0.194444,-0.222222,-0.205357,0.5,0.666667,-0.5,0.307692,-0.344092,13480,1644.425414
13481,0.015625,0.200000,-0.666667,0.005556,0.005556,0.026786,0.5,-0.666667,0.0,-0.307692,-0.153810,13481,2353.353648
13482,0.000000,-0.200000,-0.666667,0.027778,-0.005556,0.000000,0.5,0.333333,0.5,0.692308,-0.548675,13482,2932.391791
13483,0.000000,-2.066667,0.000000,0.083333,0.094444,-0.071429,-0.5,0.333333,-1.0,-0.692308,-1.617572,13483,2121.477868


In [349]:
PROBA['price'].describe()

count    13485.000000
mean      3941.373993
std       3959.318392
min        286.740079
25%        928.375153
50%       2437.079438
75%       5287.022121
max      26550.198901
Name: price, dtype: float64

In [350]:
PROBA.loc[(PROBA['price']>19000) | (PROBA['price']<0)]

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,id,price
1665,-0.296875,-0.066667,-0.666667,-0.305556,14.488889,1.419643,0.5,0.666667,0.5,0.923077,-38.942974,1665,26550.198901


In [351]:
PROBA.loc[(PROBA['price']>18000) | (PROBA['price']<0)].describe()

,carat,depth,table,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,id,price
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,1.515625,0.346667,-0.020000,0.810000,3.765556,6.308929,-0.40000,-0.066667,0.300000,-0.123077,-15.040133,8291.600000,19810.287559
std,1.747068,1.268770,0.804018,1.041614,6.051083,10.588265,0.74162,0.760117,1.151086,1.116843,21.826469,4119.772117,3768.563549
min,-0.296875,-0.400000,-0.766667,-0.322222,-0.316667,1.276786,-1.50000,-1.000000,-1.000000,-1.846154,-38.942974,1665.000000,18045.031457
25%,-0.296875,-0.333333,-0.666667,-0.305556,1.327778,1.419643,-0.50000,-0.666667,-0.500000,-0.615385,-38.942974,7336.000000,18047.028786
50%,2.078125,-0.066667,-0.333333,1.322222,1.538889,1.491071,-0.50000,0.000000,0.500000,0.461538,0.149873,9724.000000,18196.350421
75%,2.484375,-0.066667,0.666667,1.527778,1.788889,2.116071,0.00000,0.666667,0.500000,0.461538,0.907204,10341.000000,18212.828232
max,3.609375,2.600000,1.000000,1.827778,14.488889,25.241071,0.50000,0.666667,2.000000,0.923077,1.628206,12392.000000,26550.198901
